In [ ]:
from openbabel import openbabel

<div class="alert alert-block alert-success">

# EPFL Course: CH-630 Drug Discovery

## Doctoral School EDCH

## Week 4: Exercises

</div>

 <h1 style="color:green;"> Lesson 4.2: Docking </h1>

Molecular docking is a computational technique used to predict how a small molecule (ligand) binds to a target macromolecule (protein).

As you learned in Lesson 2.2.3., docking algorithms aim to predict:

- the binding pose of the ligand in the protein's binding pocket;

- the binding affinity, i.e., the strengh of the interaction.


In this lesson, you will learn how to use AutoDock Vina, one of the most widely used docking programs in computational chemistry and drug design.

Initially, you will follow the official AutoDock Vina tutorial to learn the basic workflow. Then, you will learn how to analyse docking results. Finally, you will repeat the docking for a protein-ligand complex of your choice.

> 💡 This lesson will teach you how to perform docking simulations and interpret the results.


<h2 style="color:green;"> Step 1: Perform a docking calculation with AutoDock Vina </h2>

Performing a docking simulation involves:
- preparing both the protein and the ligand,
- defining the search space,
- and running an algorithm that explores many possible binding poses.

The scoring function then evaluates these poses to estimate how favorable each interaction is.

A wide variety of docking programs exist, each with its own sampling algorithm and scoring function.

Here, we will use the commonly used open-source tools AutoDock Vina, known for its speed and ease of use.

In the commercial domain, programs such as Glide (Schrödinger), Gold (CCDC), and MOE Dock are widely used for their accuracy, advanced scoring functions, and integration with larger modeling suites.

You will follow the tutorial of AutoDock Vina:

🔗 https://autodock-vina.readthedocs.io/en/latest/docking_basic.html

Reference: *Forli, S., Huey, R., Pique, M. E., Sanner, M. F., Goodsell, D. S., & Olson, A. J. (2016). Computational protein–ligand docking and virtual drug screening with the AutoDock suite. Nature protocols, 11(5), 905-919.*<br><br>


You will need a tool called Meeko, a tool used to prepare ligands and proteins for Vina.

It should be already installed in the `drug_discovery` conda environment.

To check Meeko is running properly:

1. Open a terminal.

2. Activate the course conda environment:
    ```bash
    conda activate drug_discovery
    ```

3. Try to run Meeko:

    `mk_prepare_receptor.py -h`


If it is not working:

1. Open a terminal.

2. Activate the course conda environment:
    ```bash
    conda activate drug_discovery
    ```

3. Install Meeko
    ```bash
    micromamba install meeko
    ```
<br>

Vina is already installed in the `drug_discovery` conda environment.

However, it might not work on all computers (like Apple Silicon ARM Macs).

Try to run Vina. In the terminal where you activated the environment, type:

`vina -h`

If Vina does not run, you can download a compatible version here: 🔗 https://vina.scripps.edu/downloads/

Once downloaded, you must call Vina using the full path to the executable. Example:
`/_path_to_folder_/autodock_vina_1_1_2_mac_catalina_64bit/bin/vina --help`

From the docking, you should have obtained a file called `1iep_ligand_vina_out.pdbqt`.

This contains the structure of the best poses and their score.

<div class="alert alert-block alert-warning"> 
While docking provides only an approximate model of binding and an approximate binding affinity, it is extremely useful for rapidly screening large compound libraries, generating hypotheses for binding modes, and guiding experimental design.
</div>

<h2 style="color:green;"> Step 2: Analysing the Docking Results </h2>

Now that we have completed the docking calculations, it is important to analyze the results. 

First, we will visualize the different binding poses generated by Vina to see how they differ from each other. We will then compare these docked poses to the crystallographic pose of the ligand in the protein, which will allow us to evaluate how well the docking reproduced the known binding mode. This provides an immediate sense of the docking performance.


Next, we will inspect the top‐ranked poses to check whether the ligand fits well within the binding pocket, forms key interactions (such as hydrogen bonds, hydrophobic contacts, π–π stacking), and avoids steric clashes. 

Visual inspection with molecular viewers like PyMOL or UCSF Chimera is essential for validating that the predicted pose makes chemical sense. It is also important to compare multiple poses, evaluate the stability of recurring interaction patterns, and ensure that scoring differences are meaningful.


To analyze the docking results, we will use the tool PacVIEW from the PacDOCK webserver: 

🔗 https://pegasus.lbic.unibo.it/pacdock/PacVIEW.html

Reference: *Carbone, J.; Ghidini, A.; Romano, A.; Gentilucci, L.; Musiani, F. PacDOCK: A Web Server for Positional Distance-Based and Interaction-Based Analysis of Docking Results. Molecules 2022, 27, 6884.*

<h3 style="color:green;"> Step 2.1: Evaluating differences in the top binding poses </h3>

To investigate differences in the binding poses and compare them to the crystallographic pose of the ligand in the protein, we will use the **Docking Results Visualisation Mode** of PacVIEW: 

https://pegasus.lbic.unibo.it/pacdock/PacVIEW_Docking_Results.html

- In Reference structure, load the experimental crystallographic pose of the ligand: `1iep_ligand.sdf`.

- In Docked structure(s), load the docking output: `1iep_ligand_vina_out.pdbqt`.

(A guide on how to use the tool can be found at: https://pegasus.lbic.unibo.it/pacdock/PacVIEW_documentation.html)

<div class="alert alert-block alert-info">

1. How similar are the top-ranked poses of the ligand to the crystallographic structure?

2. How well do you think the docking program predicted the correct binding mode?

</div>

<h3 style="color:green;"> Step 2.2: Analyse Protein-Ligand interactions </h3>

To inspect how the top‐ranked poses of the ligand fit within the binding pocket and the interactions they form with the protein residues, we will use the **Receptor-Ligand Interactions Visualisation Mode** of PacVIEW: 

https://pegasus.lbic.unibo.it/pacdock/PacVIEW_Receptor_Ligand_Interactions.html

First, we will split the output file in different files for each pose using `pybel` and `openbabel`.

In [ ]:
#function to separate binding modes from a PDBQT file
def separate_structures(input_file):
    # Read the PDBQT file as text
    with open(input_file, "r") as f:
        lines = f.readlines()

    model_lines = []
    model_count = 0

    for line in lines:
        if line.startswith("MODEL"):
            model_lines = [line]  # start new model
        elif line.startswith("ENDMDL"):
            model_lines.append(line)
            model_count += 1
            # Save the current model to a temporary PDBQT file
            temp_file = f"temp_model_{model_count}.pdbqt"
            with open(temp_file, "w") as out:
                out.writelines(model_lines)
            
            # Convert temp PDBQT to PDB using OpenBabel
            obConversion = openbabel.OBConversion()
            obConversion.SetInAndOutFormats("pdbqt", "pdb")
            mol = openbabel.OBMol()
            obConversion.ReadFile(mol, temp_file)
            obConversion.WriteFile(mol, f"pose_{model_count}.pdb")
            print(f"Saved pose_{model_count}.pdb")
            
        else:
            model_lines.append(line)

In [ ]:
input_file = "1iep_ligand_vina_out.pdbqt"

separate_structures(input_file)

Now, on PacVIEW:

- In Receptor, load the structure of the protein: `1iep_receptorH.pdb`.

- In Docked structure(s), load the docking output: `pose_1.pdb`.


To visualize the interactions, activate `Receptor-Ligand Interactions`.

(A guide on how to use the tool can be found at: https://pegasus.lbic.unibo.it/pacdock/PacVIEW_documentation.html)

You can also repeat the analysis for the other docking poses to compare the interactions across different binding modes.

<div class="alert alert-block alert-info">

1. Which interactions does the ligand form within the binding pocket? 

2. How do the interactions with the protein differ between the different binding modes?

</div>

<h2 style="color:orange;"> Exercise </h2>

Now that you performed a docking tutorial and analyzed the results, you can perform the docking protein-ligand complex of your choise.

1. Choose a protein–ligand complex from the Protein Data Bank (PDB).

2. Remove the ligand from the protein and save it separately.

3. Perform a docking simulation using AutoDock Vina to redock the ligand into the protein’s binding site.

4. Analyze the results.